# EM synapse mapping

Creating a synaptome from a morphology with spines.

# Authentication

Follow the instructions below to authenticate and select a project to work in.

The morphology-with-spines that is to serve as the input of `Synaptome` generation must be accessible from the project.

In [ ]:
import obi_auth

from obi_notebook.get_projects import get_projects
from obi_notebook.get_entities import get_entities

from entitysdk.models import CellMorphology, EMCellMesh, EMDenseReconstructionDataset, Circuit
from entitysdk import Client
from entitysdk._server_schemas import AssetLabel

token = obi_auth.get_token(environment="staging", auth_mode="daf")
project_context = get_projects(token, env="staging")

# Find a skeletonized morphology as input

We search for morphologies originating from the `IARPA MICrONS mouse`, as they are derived by skeletonization

In [ ]:
entity_client = Client(token_manager=token, project_context=project_context, environment="staging")

skeletonized_morphology = entity_client.search_entity(entity_type=CellMorphology, query={
    "subject__name": 'IARPA MICrONS mouse'
}).first()

## Get `pt_root_id`. 
Next, we have to link the skeleton to an originating cell mesh. In the absence of an activity linking the two, we match them via the identifier, `pt_root_id`.

We print the description of the neuron below. Please parse the pt root id from it and fill it in below.

In [ ]:
display(skeletonized_morphology.description)

# FILL IN BELOW
pt_root_id = 864691135476581440

# Set up task

Now create a CAVE token.

In [ ]:
import caveclient
temporary_client = caveclient.CAVEclient()
temporary_client.auth.get_new_token()

Your token needs to be pasted below.

In [ ]:
from obi_one.scientific.tasks.em_synapse_mapping import EMSynapseMappingTask, EMSynapseMappingSingleConfig
from obi_one.scientific.from_id import cell_morphology_from_id

m = cell_morphology_from_id.CellMorphologyFromID(id_str=str(skeletonized_morphology.id))
task_config = EMSynapseMappingSingleConfig(initialize=EMSynapseMappingSingleConfig.Initialize(
    spiny_neuron=m,
    pt_root_id=pt_root_id,
), coordinate_output_root="obi-one-test", cave_token="PASTE_YOUR_TOKEN_HERE")

task = EMSynapseMappingTask(config=task_config)

# We peform a quick check: Does a synaptome of this neuron already exist?

In [ ]:
found_circuits = entity_client.search_entity(entity_type=Circuit, query={
    "name__ilike": f"{pt_root_id}"
}, limit=10).all()
if len(found_circuits) > 0:
    print(f"Found the following already existing entities that may built from the selected morphology with identifier {pt_root_id}:")
    print([_circ.name for _circ in found_circuits])
    print("Are you sure you want to continue?")
else:
    task.execute(entity_client)